### Notebooks takes result Erosion GeoJSON, crops by given AOI and styles it

In [1]:
import os
import geopandas as gp
import rasterio
import rasterio.mask

from pathlib import Path

In [2]:
BASE = f"/home/{os.getenv('NB_USER')}/work"

WORKDIR = os.path.join(BASE, "notebooks/ied")
RESULTSDIR = os.path.join(BASE, "results/ied")

IMAGEPATH = os.path.join(WORKDIR, "T36UXV.tif")

In [3]:
preds = gp.read_file(os.path.join(WORKDIR, "predict_T36UXV.geojson"))
masks = gp.read_file(os.path.join(WORKDIR, "masks.geojson"))

aoi_path = os.getenv("AOI", os.path.join(WORKDIR, "aoi1_20200406.geojson"))   
# aoi1_20200406.geojson
# aoi2_20200406.geojson
# aoi3_20200406.geojson

aoi = gp.read_file(aoi_path) 

In [4]:
preds.head(2)

,geometry
0,"POLYGON ((35.13407 48.64607, 35.13407 48.64598..."
1,"POLYGON ((34.75700 48.65271, 34.75700 48.65262..."


In [5]:
if preds.crs != "epsg:32636":
    preds.to_crs("epsg:32636", inplace=True)

In [6]:
preds.head(2)

,geometry
0,"POLYGON ((657190.000 5390310.000, 657190.000 5..."
1,"POLYGON ((629400.000 5390340.000, 629400.000 5..."


#### filter very small poygons

In [7]:
# limit based on preds.describe() and 50th percentile, be careful in next experiments, limit has to be changed
limit = 4650 
preds = preds.loc[preds.area >= limit]

In [8]:
aoi

,id,geometry
0,1,"MULTIPOLYGON (((35.66062 48.75821, 35.63949 48..."


#### remove None geometries

In [9]:
masks = masks.loc[masks.geometry != None].copy()

In [10]:
aoi.to_crs(preds.crs, inplace=True)
masks.to_crs(preds.crs, inplace=True)

#### fix invalid polygons

In [11]:
preds["geometry"] = preds.apply(lambda row: row.geometry.buffer(0), axis=1)

#### intersect masks and predictions with aoi

In [12]:
preds_intersected = preds.loc[preds.intersects(aoi.geometry[0])]
preds_intersected.head(3)

,geometry
62,"POLYGON ((707440.000 5393140.000, 707450.000 5..."
64,"POLYGON ((707320.000 5393200.000, 707330.000 5..."
70,"POLYGON ((706850.000 5393330.000, 706860.000 5..."


In [13]:
masks_intersected = masks.loc[masks.intersects(aoi.geometry[0])]
masks_intersected.head(3)

,id,Data,Code,Descriptio,Comments,Image,geometry
605,None,None,NaN,None,None,None,"POLYGON ((698895.041 5399640.354, 698878.985 5..."
606,None,None,NaN,None,None,None,"POLYGON ((699023.680 5399190.838, 699046.497 5..."
607,None,None,NaN,None,None,None,"POLYGON ((699091.092 5399143.567, 699123.201 5..."


#### to epsg:4326

In [14]:
preds_intersected.to_crs("epsg:4326", inplace=True)

/opt/conda/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


#### style and save predictions for AOI

In [15]:
preds_intersected['style'] = '{"color": "#F4A460", "popup_text": "erosion"}'
preds_intersected.head(3)

,geometry,style
62,"POLYGON ((35.81695 48.65685, 35.81709 48.65684...","{""color"": ""#F4A460"", ""popup_text"": ""erosion""}"
64,"POLYGON ((35.81535 48.65742, 35.81549 48.65742...","{""color"": ""#F4A460"", ""popup_text"": ""erosion""}"
70,"POLYGON ((35.80904 48.65875, 35.80918 48.65875...","{""color"": ""#F4A460"", ""popup_text"": ""erosion""}"


In [16]:
def save_geojson(df, name, results_dir):
    print("Saving .geojson file...")

    temp_path = os.path.join(BASE, f"{results_dir}/{name}.geojson.temp") 
    os.makedirs(os.path.dirname(temp_path), exist_ok=True)

    df.to_file(temp_path, driver='GeoJSON')
    
    print(f"Generated temp file: {temp_path}")
    output_path = temp_path[:-5]
    print(f"Rename temp file: {temp_path} to {output_path}")
    os.rename(temp_path, output_path)

    print("Finished")

In [17]:
save_geojson(preds_intersected, f"{Path(aoi_path).stem}_preds", RESULTSDIR)

Saving .geojson file...
Generated temp file: /home/jovyan/work/results/ied/aoi1_20200406_preds.geojson.temp
Rename temp file: /home/jovyan/work/results/ied/aoi1_20200406_preds.geojson.temp to /home/jovyan/work/results/ied/aoi1_20200406_preds.geojson
Finished


#### crop input tif by given aoi

In [18]:
def crop(input_path, output_path, aoi, name=None, date=None):
    with rasterio.open(input_path) as src:
        aoi.to_crs(src.crs, inplace=True)
        polygon = aoi.geometry[0]
       
        out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)
        out_meta = src.meta
                
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 "nodata": 0, 
                 })
        
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with rasterio.open(output_path, "w", **out_meta) as dest:
        if name:
            dest.update_tags(name=name)
        if date:
            dest.update_tags(start_date=date, end_date=date)
        dest.write(out_image)

In [19]:
def save_tif(input_path, polygon, name, results_dir):
    print("Saving .tif file...")

    temp_path = os.path.join(BASE, f"{results_dir}/{name}.tif.temp") 
    os.makedirs(os.path.dirname(temp_path), exist_ok=True)
    
    crop(input_path, temp_path, polygon, name)
    
    output_path = temp_path[:-5]
    print(f"Rename temp file: {temp_path} to {output_path}")
    os.rename(temp_path, output_path)

    print("Finished")

In [20]:
save_tif(IMAGEPATH, aoi, f"{Path(aoi_path).stem}_cropped", RESULTSDIR)

Saving .tif file...
Rename temp file: /home/jovyan/work/results/ied/aoi1_20200406_cropped.tif.temp to /home/jovyan/work/results/ied/aoi1_20200406_cropped.tif
Finished
